In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd
import re

In [2]:
# time
def getCurrentTimeStamp():
    t = time.time()
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t)))
    return t


def printDuration(start, end):
    seconds = end - start
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    print ("%02d:%02d:%02d" % (h, m, s))

In [3]:
class DataManagement:
    def __init__(self):
        self.data = {}
    
    def setData(self,attribute, data):
        if(attribute == "name"):
            index = data.find("通过")
            data = data[4:index]
            self.data[attribute] =  data
        elif(attribute == "area" or attribute == "areaB"):
            index1 = data.find("建筑面积")
            index2 = data.find("面积")
            left = index1 if index1 > 0 else index2

            if(left != -1):
                size = data[(left+2) : (left+22)]
                areas = re.findall(r"\d+\.?\d*",size)
                self.data[attribute] = areas[0] if areas else 0
            else:
                self.data[attribute] = 0
        else:
            self.data[attribute] =  data

            
    def getData(self,attribute):
        return self.data[attribute]

In [4]:
# time
startTime =  getCurrentTimeStamp()

2019-05-20 20:37:39


In [5]:
csvFile = open("namelist.csv","w+")
writer = csv.writer(csvFile)

data = DataManagement()


header = ("name","date","title","consultPrice","currentPrice","bidCount","delayCount","applyCount","area","areaB","url")

writer.writerow(header)

89

In [6]:
# tianjin
# sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
#                  "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  


# wuhan
sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&city=%CE%E4%BA%BA&sorder=2" \
                  "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-19&spm=a213w.3064813.9001.2"

In [7]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [8]:
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))


<selenium.webdriver.remote.webelement.WebElement (session="6debe61ac75f9d4eb90913a5136a1fcc", element="0.6162222530317452-1")>

In [9]:
#if test, set  pages = 1 
#pages = 1
pages = driver.find_element_by_class_name("page-total").text

In [10]:
for page in range(1,int(pages)+1):
    if(page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
    jsonData =  json.loads(driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML'))
    items = jsonData["data"]
    
    itemLen = len(items)
    #itemLen = 6   #test
    for item in range(itemLen):            
        if(items[item]["status"] == "done"):            
            data.setData("id",items[item]["id"])
            data.setData("title",items[item]["title"])
            data.setData("consultPrice",items[item]["consultPrice"])
            data.setData("currentPrice",items[item]["currentPrice"])
            data.setData("bidCount",items[item]["bidCount"])
            data.setData("delayCount",items[item]["delayCount"])
            data.setData("applyCount",items[item]["applyCount"])
            

            url = "#pai-item-"+str(data.getData("id"))
                        
            # Click the link which opens in a new window
            driver.find_element_by_css_selector(url).click()
            time.sleep(1) # have to wait sometime

            # Wait for the new window or tab
            wait.until(EC.number_of_windows_to_be(2))
            handles = driver.window_handles
            driver.switch_to.window(handles[1])
            # size
            
            urlNew = driver.current_url
            data.setData("url",urlNew)

            
            url2 = str(urlNew)+"#J_Confirmation"
            driver.get(url2)
            
            
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))            
            data.setData("name",driver.find_element_by_class_name("c-content").text)
            data.setData("date",driver.find_element_by_class_name("c-stamp").text)

           # get area
            driver.get(str(urlNew)+"#NoticeDetail")
            time.sleep(1)
            data.setData("area",driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent'))

           ###  Tip
            driver.get(str(urlNew)+"#RemindTip")
            time.sleep(1)
            data.setData("areaB",driver.find_element_by_id("J_desc").get_attribute('textContent'))
                

### write into file
            writer.writerow((data.getData("name"),
                                   data.getData("date"),
                                   data.getData("title"),
                                   data.getData("consultPrice"),
                                   data.getData("currentPrice"),
                                   data.getData("bidCount"),
                                   data.getData("delayCount"),
                                   data.getData("applyCount"),
                                   data.getData("area"),
                                   data.getData("areaB"),
                                   data.getData("url")
                                 ))
          
            driver.close()
            driver.switch_to.window(handles[0])
        else:
            pass
           # print("***item failed******"+": Page :"+str(page)+"   Item :"+str(item))



In [11]:
driver.quit()

In [12]:
csvFile.close()
csv = pd.read_csv('namelist.csv', encoding='utf-8')
csv.to_excel('namelist.xlsx', sheet_name='data')

In [13]:
endTime = getCurrentTimeStamp()

printDuration(startTime, endTime)

2019-05-20 21:28:27
00:50:48


In [14]:
# match = r"\d+\.\d*"

# areas = re.findall(match,"abc12121223ac-1.12dddd")

# areas

